# Prophet Parameter Tuning for Metrics Forecasting

This notebook helps you:
1. Query historical metrics from Victoria Metrics
2. Test different Prophet parameters
3. Visualize forecast results
4. Determine optimal parameters for each selector/job

Use this to tune parameters before adding them to `vm_forecast_config` table.


## 1. Setup and Configuration


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta, timezone, date
from prophet import Prophet
from prometheus_api_client import PrometheusConnect
import warnings
import json
warnings.filterwarnings('ignore')

# Set plot style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (15, 8)


In [ ]:
# Configuration
VM_QUERY_URL = "http://victoria-metrics:8428"  # Your Victoria Metrics URL
VM_TOKEN = ""  # Your VM token (if needed)

# Query parameters
SELECTOR = '{job="extractor"}'  # PromQL selector - CHANGE THIS
HISTORY_DAYS = 365  # Days of history to fetch
FORECAST_DAYS = 20  # Business days to forecast

print(f"Selector: {SELECTOR}")
print(f"History: {HISTORY_DAYS} days")
print(f"Forecast: {FORECAST_DAYS} business days")


## 2. Connect to Victoria Metrics and Fetch Data


In [ ]:
# Connect to Victoria Metrics
headers = {}
if VM_TOKEN:
    headers["Authorization"] = f"Bearer {VM_TOKEN}"

prom = PrometheusConnect(url=VM_QUERY_URL, headers=headers, disable_ssl=True)
print(f"Connected to Victoria Metrics at {VM_QUERY_URL}")

# Calculate date range
end_date = datetime.now(timezone.utc)
start_date = end_date - timedelta(days=HISTORY_DAYS)

print(f"Query range: {start_date.date()} to {end_date.date()}")

# Query data
print(f"\nQuerying: {SELECTOR}")
query_result = prom.custom_query_range(
    query=SELECTOR.replace("'", '"'),  # Ensure double quotes
    start_time=start_date,
    end_time=end_date,
    step="24h"
)

print(f"Query returned {len(query_result)} series")


## 3. Parse and Prepare Data


In [ ]:
def parse_series(query_result, series_index=0):
    """Parse Prometheus query result into DataFrame."""
    if not query_result or len(query_result) <= series_index:
        print(f"No data found at index {series_index}")
        return None, None
    
    item = query_result[series_index]
    metric = item.get('metric', {})
    metric_name = metric.get('__name__')
    labels = {k: v for k, v in metric.items() if k != '__name__'}
    
    values = item.get('values', [])
    
    # Convert to DataFrame
    data = []
    for ts, value in values:
        dt = datetime.fromtimestamp(float(ts), tz=timezone.utc)
        data.append({'ds': dt, 'y': float(value)})
    
    df = pd.DataFrame(data)
    
    return df, {'metric_name': metric_name, 'labels': labels}

def prepare_business_day_data(df):
    """Convert to business-day indexed data with interpolation."""
    df = df.copy()
    df['ds'] = pd.to_datetime(df['ds'], utc=True).dt.tz_localize(None)
    df['date'] = df['ds'].dt.date
    
    # Keep latest value per day
    daily = df.groupby('date').agg({'ds': 'max', 'y': 'last'}).reset_index(drop=True)
    daily['ds'] = pd.to_datetime(daily['ds'])
    
    # Create business day range
    start = daily['ds'].min()
    end = daily['ds'].max()
    all_business_days = pd.bdate_range(start=start, end=end)
    
    # Reindex to business days and interpolate
    daily = (
        daily.set_index('ds')
        .reindex(all_business_days)
        .rename_axis('ds')
        .reset_index()
    )
    daily['y'] = daily['y'].interpolate(method='linear').ffill().bfill()
    
    return daily[['ds', 'y']]

# Parse first series (change index if you have multiple)
df_raw, series_info = parse_series(query_result, series_index=0)

if df_raw is not None:
    df_prophet = prepare_business_day_data(df_raw)
    
    print(f"Metric: {series_info['metric_name']}")
    print(f"Labels: {series_info['labels']}")
    print(f"\nData shape: {df_prophet.shape}")
    print(f"Date range: {df_prophet['ds'].min()} to {df_prophet['ds'].max()}")
    print(f"Missing values: {df_prophet['y'].isna().sum()}")
    display(df_prophet.head())


In [ ]:
# Plot historical data
plt.figure(figsize=(15, 5))
plt.plot(df_prophet['ds'], df_prophet['y'], label='Historical Data', linewidth=1.5)
plt.title(f"Historical Data: {series_info['metric_name']}")
plt.xlabel('Date')
plt.ylabel('Value')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()


## 4. Define Prophet Parameter Configurations to Test


In [ ]:
# Define parameter configurations to test
PARAM_CONFIGS = {
    'default': {
        'name': 'Default',
        'params': {
            'yearly_seasonality': True,
            'weekly_seasonality': False,
            'daily_seasonality': False,
            'seasonality_mode': 'additive',
            'changepoint_prior_scale': 0.05,
        },
        'fit': {}
    },
    'flexible_trend': {
        'name': 'Flexible Trend',
        'params': {
            'yearly_seasonality': True,
            'weekly_seasonality': False,
            'daily_seasonality': False,
            'seasonality_mode': 'additive',
            'changepoint_prior_scale': 0.5,
            'n_changepoints': 50,
        },
        'fit': {'algorithm': 'Newton'}
    },
    'rigid_trend': {
        'name': 'Rigid Trend',
        'params': {
            'yearly_seasonality': True,
            'weekly_seasonality': False,
            'daily_seasonality': False,
            'seasonality_mode': 'additive',
            'changepoint_prior_scale': 0.001,
        },
        'fit': {}
    },
    'strong_seasonality': {
        'name': 'Strong Seasonality',
        'params': {
            'yearly_seasonality': True,
            'weekly_seasonality': False,
            'daily_seasonality': False,
            'seasonality_mode': 'additive',
            'changepoint_prior_scale': 0.05,
            'seasonality_prior_scale': 50.0,
        },
        'fit': {}
    },
    'multiplicative': {
        'name': 'Multiplicative Seasonality',
        'params': {
            'yearly_seasonality': True,
            'weekly_seasonality': False,
            'daily_seasonality': False,
            'seasonality_mode': 'multiplicative',
            'changepoint_prior_scale': 0.1,
        },
        'fit': {}
    },
}

print(f"Testing {len(PARAM_CONFIGS)} different parameter configurations")
for key, config in PARAM_CONFIGS.items():
    print(f"  - {config['name']}")


In [ ]:
def generate_future_business_dates(last_date, periods):
    """Generate next N business dates."""
    future_dates = []
    current = last_date
    while len(future_dates) < periods:
        current += timedelta(days=1)
        if current.weekday() < 5:  # Monday=0, Friday=4
            future_dates.append(pd.Timestamp(current))
    return future_dates

last_history_date = df_prophet['ds'].max().date()
future_dates = generate_future_business_dates(last_history_date, FORECAST_DAYS)

print(f"Last history date: {last_history_date}")
print(f"Forecast dates: {future_dates[0].date()} to {future_dates[-1].date()}")
print(f"Number of forecast business days: {len(future_dates)}")


## 5. Train Models and Generate Forecasts


In [ ]:
# Train models and store forecasts
forecasts = {}
models = {}

for config_key, config in PARAM_CONFIGS.items():
    print(f"\nTraining: {config['name']}...")
    print(f"  Parameters: {config['params']}")
    
    try:
        # Create model
        model = Prophet(**config['params'])
        
        # Fit model
        model.fit(df_prophet, **config['fit'])
        
        # Generate forecast
        future_df = pd.DataFrame({'ds': future_dates})
        forecast = model.predict(future_df)
        
        # Store results
        forecasts[config_key] = forecast
        models[config_key] = model
        
        print(f"  ✓ Success")
        
    except Exception as e:
        print(f"  ✗ Failed: {e}")
        forecasts[config_key] = None

print(f"\n{len([f for f in forecasts.values() if f is not None])}/{len(PARAM_CONFIGS)} models trained successfully")


## 6. Compare All Forecasts Visually


In [ ]:
plt.figure(figsize=(18, 10))

# Plot historical data
plt.plot(df_prophet['ds'], df_prophet['y'], 
         'ko-', label='Historical Data', linewidth=2, markersize=3, alpha=0.6)

# Plot each forecast
colors = plt.cm.tab10(np.linspace(0, 1, len(forecasts)))

for idx, (config_key, forecast) in enumerate(forecasts.items()):
    if forecast is None:
        continue
    
    config = PARAM_CONFIGS[config_key]
    color = colors[idx]
    
    # Plot trend
    plt.plot(forecast['ds'], forecast['yhat'], 
             label=f"{config['name']} (trend)", 
             linewidth=2.5, color=color, linestyle='--')
    
    # Plot uncertainty interval
    plt.fill_between(forecast['ds'], 
                     forecast['yhat_lower'], 
                     forecast['yhat_upper'],
                     alpha=0.15, color=color)

plt.axvline(x=last_history_date, color='red', linestyle=':', 
            linewidth=2, label='Forecast Start', alpha=0.7)

plt.title(f"Prophet Forecast Comparison: {series_info['metric_name']}", fontsize=16, fontweight='bold')
plt.xlabel('Date', fontsize=12)
plt.ylabel('Value', fontsize=12)
plt.legend(loc='best', fontsize=10)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()


## 7. Compare Forecast Statistics


In [ ]:
# Build comparison table
comparison_data = []

for config_key, forecast in forecasts.items():
    if forecast is None:
        continue
    
    config = PARAM_CONFIGS[config_key]
    
    comparison_data.append({
        'Configuration': config['name'],
        'Trend Mean': forecast['yhat'].mean(),
        'Trend Std': forecast['yhat'].std(),
        'Lower Mean': forecast['yhat_lower'].mean(),
        'Upper Mean': forecast['yhat_upper'].mean(),
        'Uncertainty Width': (forecast['yhat_upper'] - forecast['yhat_lower']).mean(),
        'Changepoint Scale': config['params'].get('changepoint_prior_scale', 'N/A'),
        'Seasonality Scale': config['params'].get('seasonality_prior_scale', 10.0),
        'Seasonality Mode': config['params'].get('seasonality_mode', 'additive'),
    })

comparison_df = pd.DataFrame(comparison_data)
display(comparison_df.round(2))


## 8. Generate SQL for Database Configuration

Based on your analysis, generate SQL to add the best config to the database:


In [ ]:
# Choose the configuration you want to use
selected_config = 'default'  # ← CHANGE THIS to your preferred config
JOB_ID = 'metrics_forecast'  # ← CHANGE THIS to your scheduler job_id

config = PARAM_CONFIGS[selected_config]

# Generate SQL
prophet_params_json = json.dumps(config['params'])
prophet_fit_json = json.dumps(config['fit']) if config['fit'] else None

# Build SQL statement (aligned with current schema: job_id + selection_value, no selection_type)
sql = f"""-- Add forecast configuration for {SELECTOR}
-- Note: This config will be used by job_id='{JOB_ID}'
INSERT INTO vm_forecast_config (
    job_id,
    selection_value,
    prophet_params,
    prophet_fit_params,
    notes,
    enabled
) VALUES (
    '{JOB_ID}',
    '{SELECTOR}',
    '{prophet_params_json}'::jsonb,
    {f"'{prophet_fit_json}'::jsonb" if prophet_fit_json else 'NULL'},
    'Tuned using prophet_parameter_tuning.ipynb on {datetime.now().date()}',
    true
);
"""

print("Copy this SQL to add your chosen configuration to the database:\n")
print(sql)
print("\n" + "="*80)
print(f"Job ID: {JOB_ID}")
print(f"Selector: {SELECTOR}")
print(f"Configuration: {config['name']}")
print(f"Parameters: {json.dumps(config['params'], indent=2)}")
if config['fit']:
    print(f"Fit params: {json.dumps(config['fit'], indent=2)}")


## 9. Parameter Tuning Guide

### Key Parameters:

#### **`changepoint_prior_scale`** (Trend Flexibility)
- **0.001-0.01**: Very smooth, rigid trend (stable metrics)
- **0.05**: Default, moderate flexibility
- **0.1-0.5**: Flexible, follows data closely (volatile metrics)
- **0.5+**: Very flexible, may overfit (use with caution)

#### **`seasonality_prior_scale`** (Seasonality Strength)
- **0.01-1.0**: Weak seasonal patterns
- **10.0**: Default, moderate seasonality
- **50-100**: Strong seasonal patterns

#### **`seasonality_mode`**
- **`additive`**: Seasonal effects don't scale with trend (most cases)
- **`multiplicative`**: Seasonal effects scale with trend (growing businesses)

### What to Look For:

✅ **Good forecast**: Follows historical pattern, reasonable uncertainty bands, captures seasonality  
❌ **Overfit**: Too many changepoints, erratic trend, follows noise  
❌ **Underfit**: Misses obvious patterns, too smooth, doesn't capture seasonality  

### Workflow:

1. Start with `default` configuration
2. If trend is too smooth → increase `changepoint_prior_scale`
3. If trend is too noisy → decrease `changepoint_prior_scale`
4. If seasonal patterns are weak → increase `seasonality_prior_scale`
5. If metrics grow and patterns scale → use `multiplicative` mode
6. Generate SQL and add to database


## 10. Test Multiple Series (if query returns multiple)


In [ ]:
# Show all available series from query
print(f"Available series: {len(query_result)}\n")

for idx, item in enumerate(query_result[:10]):  # Limit to first 10
    metric = item.get('metric', {})
    metric_name = metric.get('__name__')
    labels = {k: v for k, v in metric.items() if k != '__name__'}
    print(f"{idx}: {metric_name} {labels}")
    
print(f"\n💡 To test a different series, change SERIES_INDEX below and re-run from cell 7")
